In [290]:
ls

Maze.ipynb      bigMaze.txt     openMaze.txt
Untitled.ipynb  mediumMaze.txt  testMaze.txt


In [1]:
import numpy as np

In [2]:
mediumMaze = []
bigMaze = []
openMaze = []
testMaze = []

In [3]:
with open("openMaze.txt") as f:
    for line in f:
        openMaze.append(line.rstrip())
print(len(openMaze[0]), len(openMaze))
openMaze
# start = (23,1)
# end = (9,18)

37 20


['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '%                     %P            %',
 '%                     %             %',
 '%                     %             %',
 '%                     %             %',
 '%                     %             %',
 '%                     %%%%%%%%      %',
 '%                            %      %',
 '%                            %      %',
 '%                            %      %',
 '%                            %      %',
 '%                            %      %',
 '%      %%%%%%%%%%%%%%%%%            %',
 '%      %                            %',
 '%      %                            %',
 '%      %                            %',
 '%      %                            %',
 '%      %%%%                         %',
 '%        .%                         %',
 '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%']

In [4]:
with open("mediumMaze.txt") as f:
    for line in f:
        mediumMaze.append(line.rstrip())
mediumMaze
# start = (1,1)
# end = (59,21)

['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '%P    %       %             %         %     %           %   %',
 '%%% %%% % %%%%% %%%%% %%%%% %%% %%%%%%% % %%% % %%%%%%% % %%%',
 '%   %         %   % %   % %   %         %   % %     % % %   %',
 '% %%% %%% %%% %%% % %%% % %%% % %%%%%% %%%% % %%%%% % % %%% %',
 '% %   % %   % %   %   % %                 % %   % % %       %',
 '%   %%% % % % % % % % % %%% % %%%%%%% %%% % %%% % % %%%%%%% %',
 '% %   %   % %   %   % %   % %     %   %   %     % %   %   % %',
 '% %%%%%%% % % %%% %%% %%% % %%% %%% % % %%%%%%%%% %%% % % % %',
 '%   %   % %       % % %   %   % %   % % %     %     %   % % %',
 '% % %%% % %%% %%%%% % % %%%%% % % %%% % % %%% %%%%% %%%%% %%%',
 '% %   % %   %         % %   % %     %   % % %           %   %',
 '% %%%%% %%%%% %%%%% %%% % % % %%%%%%%%% % % %%% %%% %%% %%% %',
 '%     %       %         % % %         %   %     %     %   % %',
 '% %%% %%% %%%%% %%%%% % %%% %%% %%%%% %%%%%% %%%% %%% %%% % %',
 '%   %   

In [5]:
with open("bigMaze.txt") as f:
    for line in f:
        bigMaze.append(line.rstrip())
print(len(bigMaze[0]), len(bigMaze))
bigMaze
# start = (1,29)
# end = (79,1)

81 31


['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '% %           %                       %   %     %       %   %               %  .%',
 '% % %%% %%% %%% %% %%%%%%%%%% %%%%%%% %%% % %%% % % %%% % % % %%%%% %%% %%%%% % %',
 '%   %     %   % %   %       %   %   %   % % %   % % %     % %     % % % %     % %',
 '% %%%%    %%% % %%% % % %%% %%% % % %%% % % % %%% % %%%%% % %     % % % % %%%%% %',
 '%         %   %   %   % % % %   % %       %     %       % % % %   % %   % %     %',
 '%%% %% %% % %%%%% % %%% % % %  %% % %%%% %% %%% % %%%%% % % % % %%% % %%% % %%% %',
 '%       % % %   % % % % %   %     % %       % %   %   %   %     %   % %   % % % %',
 '% %%%%%%% % % % % % % % %%% % %%%%% % %%%%%%% %%%%%%% %%%%%%% %%% %%% % %%% % % %',
 '%     %   %   %   % % %   %   %           %     %   %   %         %   %   % % % %',
 '%%%%% % %%%%%   %%% % %%% % %%% %%%     % % %%% % % %%% % %%%%%%% %%% %%% % % % %',
 '%         %     %   % %   % %   %   %   %   %     % 

In [7]:
from collections import deque

In [8]:
import copy
import numpy as np
import numpy.linalg as la

In [51]:
class maze():
    
#   create a maze with list;
    def __init__(self, maze="mediumMaze.txt"):
        self.cost = {}
        self.raw_list = []
        self.maze_list = []
        self.width = None
        self.height = None
        self.end = [] # ready for multiple dots
        self.start = None
        self.visited = [] 
        self.solution = [] # ready for draw;
        
        with open(maze) as f:
            for line in f:
                self.raw_list.append(line.rstrip())
        self.width = len(self.raw_list[0])
        self.height = len(self.raw_list)
        
        for line in self.raw_list:
            for char in line:
                if char == "%":
                    self.maze_list.append(0)
                elif char == ".":
                    self.maze_list.append("G")
                    self.end.append((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))
                elif char == "P":
                    self.maze_list.append("P")
                    self.start = ((len(self.maze_list)%self.width-1, len(self.maze_list)//self.width))
                else:
                    self.maze_list.append(1)

#   get the index in the raw_list;
    def getPoint(self, x, y):
        if x < self.width and y < self.height:
            return y*self.width + x
        else:
            print("Index out of bound!")
            return False
    
#   check the wall in the maze_list;
    def accessMaze(self, x, y):
        return self.maze_list[self.getPoint(x, y)]
    
# #     right:0
# #     bottom:1
# #     left:2
# #     top:3
# # Check whether this direction can travel;
#     def canTravel(self, pos, direction):
# #         print(pos, direction,visited)
#         x, y = pos
#         if direction == 0:
#             if self.accessMaze(x+1, y) and self.getPoint(x+1, y) not in self.visited:
#                 return True
#         elif direction == 1:
#             if self.accessMaze(x, y+1) and self.getPoint(x, y+1) not in self.visited:
#                 return True
#         elif direction == 2:
#             if self.accessMaze(x-1, y) and self.getPoint(x-1, y) not in self.visited:
#                 return True
#         elif direction == 3:
#             if self.accessMaze(x, y-1) and self.getPoint(x, y-1) not in self.visited:
#                 return True
#         else:
#             return False
    
# The adjacant node who is unvisited and valid
    def adjacentList(self, pos):
        adjacent = []
        x, y = pos
        if self.accessMaze(x+1, y) and self.getPoint(x+1, y) not in self.visited:
            adjacent.append((x+1, y))
        if self.accessMaze(x, y+1) and self.getPoint(x, y+1) not in self.visited:
            adjacent.append((x, y+1))
        if self.accessMaze(x-1, y) and self.getPoint(x-1, y) not in self.visited:
            adjacent.append((x-1, y))
        if self.accessMaze(x, y-1) and self.getPoint(x, y-1) not in self.visited:
            adjacent.append((x, y-1))
        return adjacent
    
    def dfs(self, start=(1,1), end=(59,21)):
        self.visited = []
        self.solution = []
        self.visited.append(self.getPoint(start[0], start[1]))
        self.solution.append(start)
        #print("start")
        self.dfs_helper(start)
        
        
    def dfs_helper(self, current):
        adjacent_nodes = self.adjacentList(current)  
        
        if current in self.end:
            self.solution.append(current)
            #print("I get the destination!")
            return True
        
        if not adjacent_nodes:
            # Remove it from the visited list
            self.solution = self.solution[:-1]
            return
        
        for node in adjacent_nodes:
            nextStep= node
            self.visited.append(self.getPoint(node[0], node[1]))
            self.solution.append((node[0], node[1]))
            self.dfs_helper(nextStep)

        if self.solution[-1] not in self.end:
            self.solution = self.solution[:-1]       

        return
            
        
    def bfs(self, start=(1,1), end=(59,21)):
        queue = []
        self.visited = []
        self.solution = []
        parent = {}
        
        queue.append(start)
        self.visited.append(self.getPoint(start[0], start[1]))
                
        while queue:
            current = queue[0]
            queue = queue[1:]
            
            if current in self.end:
                #print("I get to the destination!")
                # The last step to the solution, track back
                self.solution.append(current)
                current = parent[current]
                while parent[current] != self.start:
                    self.solution.append(current)
                    current = parent[current]
                self.solution.append(current)
                self.solution = self.solution[::-1]
                return True
            
            adjacent_nodes = self.adjacentList(current)
            for node in adjacent_nodes:
                parent[node] = current
                queue.append(node)
                self.visited.append(self.getPoint(node[0], node[1]))     
            
        print("I can't find the destination!")
        return False
    
    # Heuristics function for single dot maze
#     def heurisFunc(self, current):
#         delta_x = current[0] - self.end[0][0]
#         delta_y = current[1] - self.end[0][1]
#         return delta_x + delta_y
        
    def A_star_search(self, start=(1,1), end=(59,21)):
        self.countCost()
        self.visited = []
        self.solution = []
        queue = []
        parent = {}
        queue.append((start[0], start[1]))
        self.visited.append(self.getPoint(start[0], start[1]))
        self.solution.append((start[0], start[1]))
        
        while queue:
            current = queue[0]
            queue = queue[1:]
            
            if current in self.end:
                #print("I get to the destination!")
                # The last step to the solution, track back
                self.solution.append(current)
                current = parent[current]
                while parent[current] != self.start:
                    self.solution.append(current)
                    current = parent[current]
                self.solution.append(current)
                self.solution = self.solution[::-1]
                return True
            
            adjacent_nodes = self.adjacentList(current)
            for node in adjacent_nodes:
                parent[node] = current
                queue.append(node)
                self.visited.append(self.getPoint(node[0], node[1]))
            queue = self.AStarSort(queue, current)
            
        print("I can't find the destination!")
        return False
        
    def AStarSort(self, queue, current):
        distance = []
        sorted_queue = []
        for node in queue:
            distance.append(self.cost[node] + self.manhattanDistance(current, node))
          
        # get the index in term of sorting
        sorted_idx = np.argsort(distance) 
        sorted_arr = (np.array(queue)[sorted_idx]) 
        for node in sorted_arr:
            sorted_queue.append(tuple(node))
            
        return sorted_queue
    
    def best_first_search(self, start=(1,1), end=(59,21)):
        self.countCost()
        self.visited = []
        self.solution = []
        self.visited.append(self.getPoint(start[0], start[1]))
        self.solution.append((start[0], start[1]))
        self.greedyHelper(start)
    
    def greedyHelper(self, current): # Almost the same as dfs
        if current in self.end:
            #print("I get the destination!")
            self.solution.append(current)
            return True
        
        adjacent_nodes = self.adjacentList(current)
        #print(adjacent_nodes)
        #print("----")
        if not adjacent_nodes:
            self.solution = self.solution[:-1]
            return
        
        # Sort the adjacent node list
        sorted_adj_nodes = self.greedySort(adjacent_nodes)
        #print(sorted_adj_nodes)
        #print("----")
        for node in sorted_adj_nodes:
            nextStep = tuple(node)
            self.visited.append(self.getPoint(node[0], node[1]))
            self.solution.append((node[0], node[1]))
            self.greedyHelper(nextStep)
            
        if self.solution[-1] not in self.end:
            self.solution = self.solution[:-1]       
        
        return
        
    # Sort the adjaceny nodes list based on Manhattan Distance
    # ** Single dots implementation **
    def greedySort(self, adjacentList):
        distance = []
        for node in adjacentList:
            distance.append(self.cost[node])
          
        # get the index in term of sorting
        sorted_idx = np.argsort(distance) 
        
        # sort the original adjacent node list, ****return a numpy array!****
        return np.array(adjacentList)[sorted_idx]     
    
    def manhattanDistance(self, cur, des): return abs(cur[0] - des[0]) + abs(cur[1] - des[1])
    
    def countCost(self):
        self.cost = {}
        for g in self.end:
            for index, cell in enumerate(self.maze_list):
                x = index%self.width
                y = index//self.width
                if not cell:
                    self.cost[(x,y)] = np.inf
                else:
                    self.cost[(x,y)] = self.manhattanDistance((x,y), g)
    
    def drawSolution(self):
        solution = copy.deepcopy(self.raw_list)  
        for pos in self.solution:
            x, y = pos
            if pos in self.end:
                solution[y] = solution[y][:x] + "G" + solution[y][x+1:]
            elif pos == self.start:
                solution[y] = solution[y][:x] + "P" + solution[y][x+1:]
            else:
                solution[y] = solution[y][:x] + "." + solution[y][x+1:]
        return solution


In [52]:
m = maze("bigMaze.txt")
#%timeit m.dfs(start=m.start, end=m.end[0])
#%timeit m.bfs(start=m.start, end=m.end[0])
#%timeit m.best_first_search(start=m.start, end=m.end[0])
%timeit m.A_star_search(start=m.start, end=m.end[0])

m.drawSolution()

26.5 ms ± 5.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '% %    .....  %.......................%   %.....%.......%...%...........    %..G%',
 '% % %%%.%%%.%%%.%% %%%%%%%%%% %%%%%%%.%%% %.%%%.%.% %%%.%.%.%.%%%%% %%%.%%%%%.% %',
 '%   %  ...%...%.%   %       %   %   %...% %.%...%.% %  ...%.%.    % % %.%.....% %',
 '% %%%%   .%%%.%.%%% % % %%% %%% % % %%%.% %.%.%%%.% %%%%% %.%.    % % %.%.%%%%% %',
 '%        .%...%...%   % % % %   % %    .. %. ...%.      % %.%.%   % %...%.%     %',
 '%%% %% %%.%.%%%%%.% %%% % % %  %% % %%%%.%%.%%%.%.%%%%% % %.%.% %%% %.%%%.% %%% %',
 '%       %.%.%...%.% % % %   %     % %   ....% %...%   %   %...  %   %.%...% % % %',
 '% %%%%%%%.%.%.%.%.% % % %%% % %%%%% % %%%%%%% %%%%%%% %%%%%%% %%% %%%.%.%%% % % %',
 '%     %...%...%...% % %   %   %           %     %   %   %         %  .%...% % % %',
 '%%%%% %.%%%%%   %%% % %%% % %%% %%%     % % %%% % % %%% % %%%%%%% %%%.%%%.% % % %',
 '%      ...%     %   % %   % %   %   %   %   %     % 

In [53]:
m = maze("mediumMaze.txt")
#%timeit m.dfs(start=m.start, end=m.end[0])
#%timeit m.bfs(start=m.start, end=m.end[0])
#%timeit m.best_first_search(start=m.start, end=m.end[0])
%timeit m.A_star_search(start=m.start, end=m.end[0])

m.drawSolution()

9.87 ms ± 758 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '%P..  %       %             %         %     %           %   %',
 '%%%.%%% % %%%%% %%%%% %%%%% %%% %%%%%%% % %%% % %%%%%%% % %%%',
 '%...%.........%   % %   % %   %         %   % %     % % %   %',
 '%.%%%.%%% %%%.%%% % %%% % %%% % %%%%%% %%%% % %%%%% % % %%% %',
 '%.%...% %   %.%...%...% %                 % %   % % %       %',
 '%...%%% % % %.%.%.%.%.% %%% % %%%%%%% %%% % %%% % % %%%%%%% %',
 '% %   %   % %...%...%.%   % %     %   %   %     % %   %   % %',
 '% %%%%%%% % % %%% %%%.%%% % %%% %%% % % %%%%%%%%% %%% % % % %',
 '%   %   % %       % %.%   %   % %   % % %     %     %   % % %',
 '% % %%% % %%% %%%%% %.% %%%%% % % %%% % % %%% %%%%% %%%%% %%%',
 '% %   % %   %      ...% %   % %     %   % % %           %   %',
 '% %%%%% %%%%% %%%%%.%%% % % % %%%%%%%%% % % %%% %%% %%% %%% %',
 '%     %       %    .....% % %         %   %     %     %   % %',
 '% %%% %%% %%%%% %%%%% %.%%% %%% %%%%% %%%%%% %%%% %%% %%% % %',
 '%   %   

In [54]:
m = maze("openMaze.txt")
#%timeit m.dfs(start=m.start, end=m.end[0])
#%timeit m.bfs(start=m.start, end=m.end[0])
#%timeit m.best_first_search(start=m.start, end=m.end[0])
%timeit m.A_star_search(start=m.start, end=m.end[0])
m.drawSolution()

28.9 ms ± 3.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


['%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%',
 '%                     %P            %',
 '%                     %.            %',
 '%                     %.            %',
 '%                     %.            %',
 '%                     %........     %',
 '%                     %%%%%%%%.     %',
 '%                            %.     %',
 '%                            %.     %',
 '%                            %.     %',
 '%                            %.     %',
 '%     ...................    %.     %',
 '%     .%%%%%%%%%%%%%%%%%.   ...     %',
 '%     .%                . ...       %',
 '%     .%                ...         %',
 '%     .%                            %',
 '%     .%                            %',
 '%     .%%%%                         %',
 '%     ...G%                         %',
 '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%']